# Duck DB with the Python client

[Documentation](https://duckdb.org/docs/api/python/overview)

Clean up previous runs

In [1]:
!rm -r db
!mkdir db
!rm -r output
!mkdir output

Install necessary libraries

In [2]:
%%capture
!pip install duckdb fsspec gcsfs pandas pyarrow gcsfs jupysql duckdb-engine polars
%load_ext sql

Authenticate with gcloud CLI

In [3]:
# !gcloud auth application-default login

Register the GCS filesystem ([source](https://duckdb.org/docs/guides/python/filesystems) of this GCS connection code)

In [2]:
import duckdb
import pandas as pd
import polars as pl
from fsspec import filesystem
import sys

duckdb.register_filesystem(filesystem('gcs'))

Create a table from a remote CSV 

In [5]:
duckdb.sql("CREATE TABLE taxi_zone_lookup as FROM read_csv('gs://anna_demo/taxi_zone_lookup.csv')")

We can now query this table directly

In [6]:
duckdb.sql("FROM taxi_zone_lookup WHERE Borough = 'Manhattan'")

┌────────────┬───────────┬───────────────────────────┬──────────────┐
│ LocationID │  Borough  │           Zone            │ service_zone │
│   int64    │  varchar  │          varchar          │   varchar    │
├────────────┼───────────┼───────────────────────────┼──────────────┤
│          4 │ Manhattan │ Alphabet City             │ Yellow Zone  │
│         12 │ Manhattan │ Battery Park              │ Yellow Zone  │
│         13 │ Manhattan │ Battery Park City         │ Yellow Zone  │
│         24 │ Manhattan │ Bloomingdale              │ Yellow Zone  │
│         41 │ Manhattan │ Central Harlem            │ Boro Zone    │
│         42 │ Manhattan │ Central Harlem North      │ Boro Zone    │
│         43 │ Manhattan │ Central Park              │ Yellow Zone  │
│         45 │ Manhattan │ Chinatown                 │ Yellow Zone  │
│         48 │ Manhattan │ Clinton East              │ Yellow Zone  │
│         50 │ Manhattan │ Clinton West              │ Yellow Zone  │
│          · │     ·

NOTE: `duckdb.sql` is using an [in-memory DB](https://duckdb.org/docs/api/python/overview#using-an-in-memory-database) i.e. the tables are not persisted. If we want to create a [persistent DB](https://duckdb.org/docs/api/python/overview#persistent-storage), we can first create a connection.

Observe: If we're doing `duckdb.sql()` we can omit the `read_parquet/read_csv` altogether because duckdb infers it from the file format

In [7]:
con = duckdb.connect("db/my_persistent_db.db")
con.register_filesystem(filesystem('gcs'))
con.sql("CREATE TABLE taxi_zone_lookup_table as FROM 'gs://anna_demo/taxi_zone_lookup.csv'")

In [8]:
con.sql("CREATE TABLE yellow_tripdata_2024_table as FROM read_parquet('gs://anna_demo/yellow_tripdata_2024-01.parquet')")
con.sql("INSERT INTO yellow_tripdata_2024_table FROM 'gs://anna_demo/yellow_tripdata_2024-02.parquet'")


According to the [documentation](https://duckdb.org/duckdb-docs.pdf), for CSV files, files will be downloaded entirely in most cases, due to the row‑based nature of the format. 

For Parquet files, DuckDB supports partial reading, i.e., it can use a combination of the Parquet metadata and HTTP range requests to only download the parts of the file that are actually required by the query

In [9]:
duckdb.sql("""SELECT total_amount
FROM 'gs://anna_demo/yellow_tripdata_2024-02.parquet'
LIMIT 5;""")

┌──────────────┐
│ total_amount │
│    double    │
├──────────────┤
│        26.78 │
│         45.0 │
│        82.69 │
│        17.15 │
│         20.6 │
└──────────────┘

In some cases, no actual data needs to be read at all as they only require reading the metadata e.g. counting rows

In [10]:
duckdb.sql("""SELECT count(*)
FROM 'gs://anna_demo/yellow_tripdata_2024-02.parquet';""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      3007526 │
└──────────────┘

Ok, back to the DB connection we made. We can query the table we created using the `table` function

In [11]:
con.table("taxi_zone_lookup_table")

┌────────────┬───────────────┬───────────────────────────┬──────────────┐
│ LocationID │    Borough    │           Zone            │ service_zone │
│   int64    │    varchar    │          varchar          │   varchar    │
├────────────┼───────────────┼───────────────────────────┼──────────────┤
│          1 │ EWR           │ Newark Airport            │ EWR          │
│          2 │ Queens        │ Jamaica Bay               │ Boro Zone    │
│          3 │ Bronx         │ Allerton/Pelham Gardens   │ Boro Zone    │
│          4 │ Manhattan     │ Alphabet City             │ Yellow Zone  │
│          5 │ Staten Island │ Arden Heights             │ Boro Zone    │
│          6 │ Staten Island │ Arrochar/Fort Wadsworth   │ Boro Zone    │
│          7 │ Queens        │ Astoria                   │ Boro Zone    │
│          8 │ Queens        │ Astoria Park              │ Boro Zone    │
│          9 │ Queens        │ Auburndale                │ Boro Zone    │
│         10 │ Queens        │ Baisley

We can also check the columns with `describe` (there are probably more interesting functions but I haven't explored much)

In [12]:
con.table("yellow_tripdata_2024_table").describe()

┌─────────┬─────────────────────┬──────────────────────┬───┬──────────────────────┬────────────────────┐
│  aggr   │      VendorID       │ tpep_pickup_datetime │ … │ congestion_surcharge │    Airport_fee     │
│ varchar │       double        │       varchar        │   │        double        │       double       │
├─────────┼─────────────────────┼──────────────────────┼───┼──────────────────────┼────────────────────┤
│ count   │           5972150.0 │ 5972150              │ … │            5646378.0 │          5646378.0 │
│ mean    │   1.754722503620974 │ NULL                 │ … │    2.261295382632902 │  0.135142962444243 │
│ stddev  │ 0.43126231634052253 │ NULL                 │ … │   0.8183577430533403 │ 0.4783123471983139 │
│ min     │                 1.0 │ 2002-12-31 22:59:39  │ … │                 -2.5 │              -1.75 │
│ max     │                 6.0 │ 2024-03-01 00:01:37  │ … │                  2.5 │               1.75 │
│ median  │                 2.0 │ NULL                 

We can connect using the DuckDB SQL extension now if we want, to see the table (we need to make sure to close the connection first otherwise it will be locked)

In [13]:
con.close()

We can also connect directly from the jupyter sql magic extension which we loaded above

In [14]:
%%sql duckdb:///db/my_persistent_db.db
SELECT DISTINCT Zone FROM taxi_zone_lookup_table ORDER BY Zone LIMIT 5

Connecting to 'duckdb:///db/my_persistent_db.db'

Zone
Allerton/Pelham Gardens
Alphabet City
Arden Heights
Arrochar/Fort Wadsworth
Astoria


In [15]:
# NOTE: This is not locked by the juptyer sql magic, but we cannot use read-only here 
# because we didn't use read only in the connection with jupysql (they have to be in the same mode)
con = duckdb.connect("db/my_persistent_db.db")
con.sql("SELECT DISTINCT Zone FROM taxi_zone_lookup_table ORDER BY Zone LIMIT 5")

┌─────────────────────────┐
│          Zone           │
│         varchar         │
├─────────────────────────┤
│ Allerton/Pelham Gardens │
│ Alphabet City           │
│ Arden Heights           │
│ Arrochar/Fort Wadsworth │
│ Astoria                 │
└─────────────────────────┘

Let's close the connections ([source for extension](https://github.com/catherinedevlin/ipython-sql))

In [16]:
%sql -x duckdb:///db/my_persistent_db.db

In [17]:
con.close()


NOTE: If we connected with the VSCode extension, this locks it here so we would have to disconnect or say read_only

In [18]:
con = duckdb.connect("db/my_persistent_db.db", read_only=True)
con.sql("SELECT DISTINCT Zone FROM taxi_zone_lookup_table ORDER BY Zone LIMIT 5")

┌─────────────────────────┐
│          Zone           │
│         varchar         │
├─────────────────────────┤
│ Allerton/Pelham Gardens │
│ Alphabet City           │
│ Arden Heights           │
│ Arrochar/Fort Wadsworth │
│ Astoria                 │
└─────────────────────────┘

In [19]:
con.close()

We can also read a file like this ([source](https://duckdb.org/docs/data/parquet/overview#read_parquet-function))

In [20]:
duckdb.read_csv('gs://anna_demo/taxi_zone_lookup.csv')

┌────────────┬───────────────┬───────────────────────────┬──────────────┐
│ LocationID │    Borough    │           Zone            │ service_zone │
│   int64    │    varchar    │          varchar          │   varchar    │
├────────────┼───────────────┼───────────────────────────┼──────────────┤
│          1 │ EWR           │ Newark Airport            │ EWR          │
│          2 │ Queens        │ Jamaica Bay               │ Boro Zone    │
│          3 │ Bronx         │ Allerton/Pelham Gardens   │ Boro Zone    │
│          4 │ Manhattan     │ Alphabet City             │ Yellow Zone  │
│          5 │ Staten Island │ Arden Heights             │ Boro Zone    │
│          6 │ Staten Island │ Arrochar/Fort Wadsworth   │ Boro Zone    │
│          7 │ Queens        │ Astoria                   │ Boro Zone    │
│          8 │ Queens        │ Astoria Park              │ Boro Zone    │
│          9 │ Queens        │ Auburndale                │ Boro Zone    │
│         10 │ Queens        │ Baisley

Or for parquet [source](https://duckdb.org/docs/guides/file_formats/query_parquet.html)

In [21]:
duckdb.read_parquet("gs://anna_demo/yellow_tripdata_2024-01.parquet").limit(10)

┌──────────┬──────────────────────┬──────────────────────┬───┬──────────────┬──────────────────────┬─────────────┐
│ VendorID │ tpep_pickup_datetime │ tpep_dropoff_datet…  │ … │ total_amount │ congestion_surcharge │ Airport_fee │
│  int32   │      timestamp       │      timestamp       │   │    double    │        double        │   double    │
├──────────┼──────────────────────┼──────────────────────┼───┼──────────────┼──────────────────────┼─────────────┤
│        2 │ 2024-01-01 00:57:55  │ 2024-01-01 01:17:43  │ … │         22.7 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:03:00  │ 2024-01-01 00:09:36  │ … │        18.75 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:17:06  │ 2024-01-01 00:35:01  │ … │         31.3 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:36:38  │ 2024-01-01 00:44:56  │ … │         17.0 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:46:51  │ 2024-01-01 00:52:57  │ … │         16.1 │   

There's also a relational API similar to pandas/polars. Although if you're doing filters on columns on a remote file, then I think doing them in the query directly is faster because then duckdb will only retrieve the columns needed

In [22]:
duckdb.read_parquet("gs://anna_demo/yellow_tripdata_2024-01.parquet").select(
    "total_amount"
).filter("total_amount > 100").limit(5)

┌──────────────┐
│ total_amount │
│    double    │
├──────────────┤
│       127.94 │
│       134.75 │
│       103.36 │
│       114.96 │
│        217.2 │
└──────────────┘

In [23]:
duckdb.sql("""SELECT total_amount FROM 'gs://anna_demo/yellow_tripdata_2024-01.parquet'
           WHERE total_amount > 100 LIMIT 5""")

┌──────────────┐
│ total_amount │
│    double    │
├──────────────┤
│       127.94 │
│       134.75 │
│       103.36 │
│       114.96 │
│        217.2 │
└──────────────┘

We can also query data with a wildcard e.g. I have two files that match this pattern in my bucket

In [24]:
duckdb.sql("SELECT count(*) FROM 'gs://anna_demo/yellow_tripdata_2024-01.parquet'")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      2964624 │
└──────────────┘

In [25]:
duckdb.sql("SELECT count(*) FROM 'gs://anna_demo/yellow_tripdata_2024-02.parquet'")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      3007526 │
└──────────────┘

In [26]:
duckdb.sql("SELECT count(*) FROM 'gs://anna_demo/yellow_tripdata_*.parquet'")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      5972150 │
└──────────────┘

We can create a pandas dataframe from a file or table. There are other options for converting to a polars df, arrow table, numpy arrays, or list of tuples [source](https://duckdb.org/docs/api/python/overview#result-conversion))

In [27]:
taxi_zone_lookup_manhattan_df = duckdb.sql("FROM taxi_zone_lookup WHERE Borough = 'Manhattan'").df()
taxi_zone_lookup_manhattan_df

,LocationID,Borough,Zone,service_zone
0,4,Manhattan,Alphabet City,Yellow Zone
1,12,Manhattan,Battery Park,Yellow Zone
2,13,Manhattan,Battery Park City,Yellow Zone
3,24,Manhattan,Bloomingdale,Yellow Zone
4,41,Manhattan,Central Harlem,Boro Zone
...,...,...,...,...
64,246,Manhattan,West Chelsea/Hudson Yards,Yellow Zone
65,249,Manhattan,West Village,Yellow Zone
66,261,Manhattan,World Trade Center,Yellow Zone
67,262,Manhattan,Yorkville East,Yellow Zone


We can then query the pandas dataframe directly

In [28]:
duckdb.sql("FROM taxi_zone_lookup_manhattan_df")

┌────────────┬───────────┬───────────────────────────┬──────────────┐
│ LocationID │  Borough  │           Zone            │ service_zone │
│   int64    │  varchar  │          varchar          │   varchar    │
├────────────┼───────────┼───────────────────────────┼──────────────┤
│          4 │ Manhattan │ Alphabet City             │ Yellow Zone  │
│         12 │ Manhattan │ Battery Park              │ Yellow Zone  │
│         13 │ Manhattan │ Battery Park City         │ Yellow Zone  │
│         24 │ Manhattan │ Bloomingdale              │ Yellow Zone  │
│         41 │ Manhattan │ Central Harlem            │ Boro Zone    │
│         42 │ Manhattan │ Central Harlem North      │ Boro Zone    │
│         43 │ Manhattan │ Central Park              │ Yellow Zone  │
│         45 │ Manhattan │ Chinatown                 │ Yellow Zone  │
│         48 │ Manhattan │ Clinton East              │ Yellow Zone  │
│         50 │ Manhattan │ Clinton West              │ Yellow Zone  │
│          · │     ·

We can also do this without converting to a dataframe, but just assigning to a variable

In [29]:
taxi_zone_lookup_manhattan = duckdb.sql("FROM taxi_zone_lookup WHERE Borough = 'Manhattan' limit 5")
duckdb.sql("FROM taxi_zone_lookup_manhattan")

┌────────────┬───────────┬───────────────────┬──────────────┐
│ LocationID │  Borough  │       Zone        │ service_zone │
│   int64    │  varchar  │      varchar      │   varchar    │
├────────────┼───────────┼───────────────────┼──────────────┤
│          4 │ Manhattan │ Alphabet City     │ Yellow Zone  │
│         12 │ Manhattan │ Battery Park      │ Yellow Zone  │
│         13 │ Manhattan │ Battery Park City │ Yellow Zone  │
│         24 │ Manhattan │ Bloomingdale      │ Yellow Zone  │
│         41 │ Manhattan │ Central Harlem    │ Boro Zone    │
└────────────┴───────────┴───────────────────┴──────────────┘

We can also do joins on remote files e.g.

In [30]:
duckdb.sql("""SELECT l.Borough, max(total_amount) 
           FROM 'gs://anna_demo/yellow_tripdata_2024-01.parquet' t
           INNER JOIN 'gs://anna_demo/taxi_zone_lookup.csv' l
              on t.PULocationID = l.LocationID
             GROUP BY l.Borough
           ORDER BY 1
           """)

┌───────────────┬───────────────────┐
│    Borough    │ max(total_amount) │
│    varchar    │      double       │
├───────────────┼───────────────────┤
│ Bronx         │            2225.3 │
│ Brooklyn      │             801.0 │
│ EWR           │            323.33 │
│ Manhattan     │            775.48 │
│ N/A           │             900.0 │
│ Queens        │            940.93 │
│ Staten Island │            354.23 │
│ Unknown       │            5000.0 │
└───────────────┴───────────────────┘

In [31]:
duckdb.sql("""SELECT count(*)
           FROM 'gs://anna_demo/yellow_tripdata_2024-01.parquet'""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      2964624 │
└──────────────┘

In [32]:
duckdb.sql("""SELECT count(*)
           FROM 'gs://anna_demo/yellow_tripdata_2024-02.parquet'""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      3007526 │
└──────────────┘

In [33]:
duckdb.sql("""SELECT count(*)
           FROM 'gs://anna_demo/yellow_tripdata_*.parquet'""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      5972150 │
└──────────────┘

Let's read some real data from one of our [RTL buckets](https://console.cloud.google.com/storage/browser/mle_datasets_dev/rtl_publication/rtl_publication_etl/data?pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))&project=ocean-ml-sandbox). This contains the PublicationId, Title and Abstract

In [47]:
ddb_df = duckdb.sql("""FROM 'gs://mle_datasets_dev/rtl_publication/rtl_publication_etl/data/data-000000000000.parquet'""")
ddb_df_filtered = duckdb.sql("""
        FROM ddb_df
        WHERE PublicationId = 360777386437""")
ddb_df_filtered

┌───────────────┬──────────────────────┬───────────────────────────────────────────────────────────────────────────────┐
│ PublicationId │        Title         │                                   Abstract                                    │
│     int64     │       varchar        │                                    varchar                                    │
├───────────────┼──────────────────────┼───────────────────────────────────────────────────────────────────────────────┤
│  360777386437 │ preparation and pr…  │ ferroelectric lead lanthanum zirconate titanate (plzt 9/65/35) thin films w…  │
└───────────────┴──────────────────────┴───────────────────────────────────────────────────────────────────────────────┘

In [48]:
print(f"Size of the duckdb object: {sys.getsizeof(ddb_df)} bytes")
print(f"Size of the filtered duckdb object: {sys.getsizeof(ddb_df_filtered)} bytes")

Size of the duckdb object: 56 bytes
Size of the filtered duckdb object: 56 bytes


Let's compare with pandas in terms of speed and size

In [49]:
pd_df = pd.read_parquet('gs://mle_datasets_dev/rtl_publication/rtl_publication_etl/data/data-000000000000.parquet')
pd_df_filtered = pd_df[pd_df["PublicationId"] == 360777386437]
pd_df_filtered

,PublicationId,Title,Abstract
9997,360777386437,"preparation and properties of (pb, la)(zr,ti)o...",ferroelectric lead lanthanum zirconate titanat...


In [50]:
print(f"Size of the pandas dataframe object: {sys.getsizeof(pd_df)} bytes")
print(f"Size of the filtered pandas dataframe object: {sys.getsizeof(pd_df_filtered)} bytes")


Size of the pandas dataframe object: 283386133 bytes
Size of the filtered pandas dataframe object: 1834 bytes


Let's compare with polars

In [51]:
pl_df = pl.scan_parquet('gs://mle_datasets_dev/rtl_publication/rtl_publication_etl/data/data-000000000000.parquet')
pl_df_filtered = pl_df.filter(pl.col("PublicationId") == 360777386437).collect()
pl_df_filtered

PublicationId,Title,Abstract
i64,str,str
360777386437,"""preparation and properties of …","""ferroelectric lead lanthanum z…"


In [52]:
print(f"Size of the polars lazyframe object: {sys.getsizeof(pl_df)} bytes")
print(f"Size of the filtered polars dataframe object: {sys.getsizeof(pl_df_filtered)} bytes")


Size of the polars lazyframe object: 48 bytes
Size of the filtered polars dataframe object: 48 bytes


So far, polars is pretty comparable to duckdb whereas pandas obviously requires more memory

Let's see how many records we have over all the parquet files (this takes about 4mins because there are 500 files)

In [54]:
rtl_publications_count = duckdb.sql("""
        SELECT count(*)
        FROM 'gs://mle_datasets_dev/rtl_publication/rtl_publication_etl/data/data-*.parquet'""")
rtl_publications_count

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     77418681 │
└──────────────┘

This takes longer with polars (I gave up after 10mins)

In [2]:
pl_all_data = pl.scan_parquet('gs://mle_datasets_dev/rtl_publication/rtl_publication_etl/data/data-*.parquet')
pl_all_data.select(pl.len())

As we can see, Polars performance is pretty comparable. Probably just a matter of preference e.g how we want to save the data, explore the data, whether we want to use SQL or learn the polars API, etc. Nothing stopping us from using a combination of all 3. 

Personally I find duckdb more intuitive than polars since it's just SQL, but I've also never used it for a real project so I'd have to see how easy it would be to use in a production environment

# Appendix
Uncomment the below cells if you want to play around with the data
NOTE: It takes about 1h to load in all the data

In [25]:
con = duckdb.connect("db/rtl.db")
con.register_filesystem(filesystem('gcs'))

In [16]:
from google.cloud import storage

def list_blobs(bucket_name, prefix):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix)

    # Note: The call returns a response only when the iterator is consumed.
    for blob in blobs:
        yield blob.name

In [26]:
for i, filename in enumerate(list_blobs(bucket_name="mle_datasets_dev", prefix="rtl_publication/rtl_publication_etl/data")):
    if i == 0:
        action_type = "CREATE TABLE rtl_publications AS "
    else:
        action_type = "INSERT INTO rtl_publications "
    
    con.sql(f"""{action_type} FROM 'gs://mle_datasets_dev/{filename}'""")
    print("Done with", filename)

Done with rtl_publication/rtl_publication_etl/data/data-000000000000.parquet
Done with rtl_publication/rtl_publication_etl/data/data-000000000001.parquet
Done with rtl_publication/rtl_publication_etl/data/data-000000000002.parquet
Done with rtl_publication/rtl_publication_etl/data/data-000000000003.parquet
Done with rtl_publication/rtl_publication_etl/data/data-000000000004.parquet
Done with rtl_publication/rtl_publication_etl/data/data-000000000005.parquet
Done with rtl_publication/rtl_publication_etl/data/data-000000000006.parquet


In [ ]:
result_df = con.sql("FROM rtl_publications WHERE PublicationId = 360777386437")
result_df